# Numerical Panel Methods 🚢 

In the first notebook we studied

 1. _How can we calculate the influence of each panel?_ **Gaussian quadratures.**
 2. _How should we compute the derivatives of our functions?_ **Automatic differentiation.**

In this notebook we will actually start solving p-flow problems with panels methods addressing two more questions: 

 3. _How can we determine the correct strength for each panel?_ **Set-up and solve a linear system.**
 4. _How should we determine if a method is working?_ **Convergence and validation tests.** 

## 3D example: sphere

To get started, let's set up a example problem: the 3D example of the flow around a sphere.

The `sphere` function below creates a vector of panels using the parametric equation equation 

$$ [S_x,S_y,S_z] = [a\cos(θ₂)*\sin(θ₁),a\sin(θ₂)*\sin(θ₁),a\cos(θ₁)] $$

where `θ₁,θ₂` are the azimuth and polar angles.

In [ ]:
include("../src/solve.jl")
using Plots
"""
    sphere(h;a=1) -> Table(panel_props)

Sample a sphere of radius `a` such that the arc-length ≈ h in each direction.
θ₁=[0,π]: azimuth angle, θ₂=[0,2π]: polar angle.
"""
function sphere(h;a=1)
    S(θ₁,θ₂) = a .* [cos(θ₂)*sin(θ₁),sin(θ₂)*sin(θ₁),cos(θ₁)]
    dθ₁ = π/round(π*a/h)
    mapreduce(vcat,0.5dθ₁:dθ₁:π) do θ₁
        dθ₂ = 2π/round(2π*a*sin(θ₁)/h) # get polar step at this azimuth
        param_props.(S,θ₁,0.5dθ₂:dθ₂:2π,dθ₁,dθ₂)
    end |> Table
end

# Create the panels and check how many of them were made
h = 0.3; panels = sphere(h); N=length(panels)

# Get the vector of centroids (`panels.x`) and split that into  `x,y,z` vectors
x,y,z = eachrow(stack(panels.x));
plot(x,y,z,legend=false,title="Sphere represented with $N panels")

The function sets the azimuth and polar spacings so the arc length on each panel is nearly `h`. Then, it evaluates the function `S(θ₁,θ₂)` at those spacings, filling a vector with all the panel information using the `param_props` function defined in `solve.jl`. 

I've set $h=0.3$ and left the $a=1$ default. Let's check if the area's add up to 4π and they are all $\approx h^2$

In [ ]:
# Check the areas `panels.dA`
A_error = sum(panels.dA)/4π-1
A_percent = round(100A_error,digits=1)
plot(z,panels.dA/h^2,ylim=(0,2),xlabel="z",ylabel="dA/h²",
    legend=false, title="Error in total surface area=$A_percent%")

Looks good. 

Note that the number of panels is approximately $N\approx 4\pi a^2/h^2$. This shows that $N \sim 1/h^2$, which we might worry us as we start thinking about how to solve for the flow on these panels.

## Total flow equations

The potential of the full sphere is simply the superposition of each panel's contribution,

$$ \varphi(x) = \sum_{i=1}^N q_i \phi_i(x) $$

where $N$ is the number of panels, $q$ is the vector of **unknown** panel strengths and $\phi_i$ is influence of panel $i$.

Since we used a $G$ satisfying $\nabla^2 G=0$ and this equation is linear, _any_ vector $q$ will satisfy the laplace equation $\nabla^2\varphi=0$ and be a valid potential flow. This is nice since we can't mess that up, but it means we need an additional equation to determine the _correct_ $q$ for a given geometry and flow condition.

 3. _How can we determine the correct $q$ for each panel?_ 

## Apply boundary conditions

The additional equations from our problem description are the boundary conditions. Defining $\vec U$ as the free stream velocity and $\hat n$ as the surface normal, the conditions in an infinite fluid (no free surface) are
 - Flow tangency on the solid body's surface: $U_n+u_n = U_n+\frac{\partial\varphi}{\partial n}=0$
 - No disturbance far from the body: $u(\infty)\rightarrow 0$ 

The second condition is achieved automatically since $u(r) \sim \frac{\partial G}{\partial r} = 1/r^2$. Therefore, the first condition must be used to set $q$.

Substituting the equation for $\varphi$ into the body BC, we have

$$ \vec U \cdot \hat n(\vec S) + \sum_{i=1}^N \frac{\partial\phi_i}{\partial n}(\vec S) q_i = 0$$

This boundary condition is linear in $q$ and applies to every point on the body surface. Applying the boundary condition at $N$ specific locations will create $N$ linear equations of the $N$ unknown components of $q$. We will choose the centroid $\vec c$ of each panel to apply this condition. Defining 

$$ a_{i,j} = \frac{\partial\phi_i}{\partial n}(\vec c_j), \quad b_j = -\vec U \cdot \hat n(\vec c_j) $$

as the component of the influence matrix $A$ and the excitation vector $b$, we have

$$\sum_{i=1}^N a_{i,j} q_i = b_j \quad j=1\ldots N$$

or simply $Aq = b$. So, _how can we determine the correct $q$ for each panel?_ Construct $A,b$ and type

```julia
q = A\b
```